Multi-Driver Lap Delta Telemetry Analysis

This notebook engineers telemetry data from the Fast-F1 API for multiple drivers to compute lap time delta traces. It utilizes a structured workflow for data acquisition, alignment, and delta calculation, producing visualizations that identify where time is gained or lost across a lap. Additional stacked telemetry plots (speed, throttle, brake, RPM, etc.) provide context for performance differences between drivers.

NOTE: This notebook will not compile as standalone. Please pull down the project from GitHub (git clone https://github.com/yourusername/f1-driving-style-analytics-tool.git) and install the necessary dependencies. Further instructions are included in the README.md.

The code below adds the parent directory to Python’s module search path and configures logging to suppress all FastF1 logs below the warning level. This will enable subsequent code blocks that use imports to work seamlessly and keep my resulting code compilations clean and easy to read.

In [1]:
import sys
import os
import logging

root = os.path.abspath("..")
sys.path.append(root)

In this section, I import Python libraries for data visualization, numerical analysis, and working with the Pandas dataframes that the FastF1 API is primarily structured with. I also import custom functions and modules for preprocessing F1 data and constants. To support full visibility into the datasets without truncation, I configure Pandas display options to show all rows and columns.

In [2]:
from src.data import f1_data
from src.utils import f1_constants
from src.preprocessing import telemetry_cleaning, telemetry_processing

import pandas as pd
import numpy as np
import plotly.graph_objects as go

pd.set_option('display.max_rows', None) # reset_option to compact dataframe view
pd.set_option('display.max_columns', None)

The following code initializes a single F1 race session by defining parameters such as year, location, and session type. These values are passed into the custom F1Session class (from f1_data.py), which creates a session object built on top of Fast-F1. This object provides access to race data as well as custom functions I’ve implemented.

These session parameters were chosen for this analysis based on the following criteria:

- Location: Melbourne (rainy conditions)
- Green Flag Laps: Remove safety car and VSC distortions
- Single-compound Windows: Avoid mixed group comparisons
- Minimize Outliers: Tire age approx. 20 laps

In [3]:
logging.getLogger('fastf1').setLevel(logging.ERROR)

year = 2025
grand_prix = f1_constants.F1Constants.LOCATIONS["Australia"]
session_type = f1_constants.F1Constants.SESSIONS["R"]

safety_car_laps = []

session = f1_data.F1Session(year, grand_prix, session_type)

All drivers who participated in the specific location's Grand Prix will be analyzed and assigned variables to be identified by their three-letter name code.

Constants for telemetry data is also initialized as variables for ease of use during visualization.

Sector timestamps variables are set for easy replacment during telemetry filtering.

In [4]:
"""DRIVER CONSTANTS"""
norris = f1_constants.F1Constants.DRIVERS["Lando Norris"]
piastri = f1_constants.F1Constants.DRIVERS["Oscar Piastri"]
verstappen = f1_constants.F1Constants.DRIVERS["Max Verstappen"]
russell = f1_constants.F1Constants.DRIVERS["George Russell"]
tsunoda = f1_constants.F1Constants.DRIVERS["Yuki Tsunoda"]
albon = f1_constants.F1Constants.DRIVERS["Alexander Albon"]
leclerc = f1_constants.F1Constants.DRIVERS["Charles Leclerc"]
hamilton = f1_constants.F1Constants.DRIVERS["Lewis Hamilton"]
gasly = f1_constants.F1Constants.DRIVERS["Pierre Gasly"]
sainz = f1_constants.F1Constants.DRIVERS["Carlos Sainz"]
hadjar = f1_constants.F1Constants.DRIVERS["Isack Hadjar"]
alonso = f1_constants.F1Constants.DRIVERS["Fernando Alonso"]
stroll = f1_constants.F1Constants.DRIVERS["Lance Stroll"]
doohan = f1_constants.F1Constants.DRIVERS["Jack Doohan"]
bortoleto = f1_constants.F1Constants.DRIVERS["Gabriel Bortoleto"]
antonelli = f1_constants.F1Constants.DRIVERS["Andrea Kimi Antonelli"]
hulkenberg = f1_constants.F1Constants.DRIVERS["Nico Hulkenberg"]
lawson = f1_constants.F1Constants.DRIVERS["Liam Lawson"]
ocon = f1_constants.F1Constants.DRIVERS["Esteban Ocon"]
bearman = f1_constants.F1Constants.DRIVERS["Oliver Bearman"]

"""THE 6 KEY TELEMETRY TRACE CONSTANTS"""
speed = f1_constants.F1Constants.TELEMETRY_COLUMNS["Speed (km/h)"] # comment out conversion and rename column in telemetry_cleaning.py
throttle = f1_constants.F1Constants.TELEMETRY_COLUMNS["Throttle (%)"]
brakes = f1_constants.F1Constants.TELEMETRY_COLUMNS["BrakesApplied"]
rpm = f1_constants.F1Constants.TELEMETRY_COLUMNS["RPM"]
gear = f1_constants.F1Constants.TELEMETRY_COLUMNS["nGear"]
# steering = f1_constants.F1Constants.TELEMETRY_COLUMNS["Steering Wheel Angle (°)"]

"""SECTOR TIMESTAMP VARIABLES"""
s1_start = 'Sector1Start'
s1_end_s2_start = 'Sector1End_Sector2Start'
s2_end_s3_start = 'Sector2End_Sector3Start'
s3_end = 'Sector3End'

This code retrieves circuit corner data (Turn, X/Y coordinates, Angle, and Distance) to place markers on stacked telemetry and delta plots.

In [5]:
corner_position = session.get_circuit_info().corners
corner_position_cleaned = telemetry_cleaning.clean_circuit_corner_data(corner_position)

critical_turn = [None]
radius = 0

Retrieve the fastest lap telemetry for Driver 1 and Driver 2. This code block processes the raw telemetry data to extract the relevant information for desired lap telemetry (any or fastest) for selected drivers.

[0][i] = sector_telemetry_list

[1][i] = driver_laps_filtered

[2][i] = sector_timestamps_dict

In [6]:
driver_1 = verstappen
driver_2 = hamilton
lap_idx = 20

"""PROCESS DRIVER 1 TELEMETRY"""
driver_1_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=driver_1,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_start,
    end=s3_end
)

driver_1_all_laps = driver_1_processed_data[1]
driver_1_telemetry = driver_1_processed_data[0][lap_idx]

"""PROCESS DRIVER 2 TELEMETRY"""
driver_2_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=driver_2,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_start,
    end=s3_end
)

driver_2_all_laps = driver_2_processed_data[1]
driver_2_telemetry = driver_2_processed_data[0][lap_idx]

This code block aligns the telemetry data for Driver 1 and Driver 2 based on distance along the lap. 

It identifies the maximum distance covered by each driver and creates a common distance array that both telemetry datasets can be interpolated onto. 

This allows for direct comparison of telemetry values at corresponding points along the lap.

In [7]:
"""NORMALIZE X-AXIS"""
max_dist_driver_1 = driver_1_telemetry['Distance (m)'].max()
max_dist_driver_2 = driver_2_telemetry['Distance (m)'].max()

x_start = 0
x_end = max(max_dist_driver_1, max_dist_driver_2)
dx = 0.1

common_distance = np.arange(x_start, x_end, dx)

"""INTERPOLATE Y-AXIS"""
y_trace = 'Speed (km/h)'
interpolate_driver_1_speed = np.interp(common_distance, driver_1_telemetry['Distance (m)'], driver_1_telemetry[y_trace])
interpolate_driver_2_speed = np.interp(common_distance, driver_2_telemetry['Distance (m)'], driver_2_telemetry[y_trace])

"""RETRIEVE APEX LOCATIONS, TURN, & LAP NUMBERS"""
apex_distances = corner_position_cleaned['Distance (1/10 m)']
turn_numbers = corner_position_cleaned['Turn']
max_speed = max(interpolate_driver_1_speed.max(), interpolate_driver_2_speed.max())
lap_number = driver_1_telemetry['LapNumber'].max()

"""COMPUTE TIME DELTA"""
driver_1_speed_ms = interpolate_driver_1_speed * 0.277778
driver_2_speed_ms = interpolate_driver_2_speed * 0.277778
driver_1_speed_ms = np.clip(driver_1_speed_ms, 1e-3, None)
driver_2_speed_ms = np.clip(driver_2_speed_ms, 1e-3, None)

dt_driver_1 = dx / driver_1_speed_ms
dt_driver_2 = dx / driver_2_speed_ms

time_driver_1 = np.cumsum(dt_driver_1)
time_driver_2 = np.cumsum(dt_driver_2)

time_delta = time_driver_2 - time_driver_1

Plot comparative telemetry traces (i.e. speed, throttle, brake, RPM) for Driver 1 and Driver 2 on the same graph to analyze performance differences across the lap. 

The x-axis is distance along the lap, and the y-axis is the telemetry value. Markers indicate corner locations for context.

In [8]:
"""ADD DRIVER TRACES"""
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=common_distance,
    y=interpolate_driver_1_speed,
    mode='lines',
    name=driver_1,
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=common_distance,
    y=interpolate_driver_2_speed,
    mode='lines',
    name=driver_2,
    line=dict(color='blue')
))

"""ADD TIME DELTA"""
fig.add_trace(go.Scatter(
    x=common_distance,
    y=time_delta,
    mode='lines',
    name='Time Delta (s)',
    line=dict(color='yellow', width=.3),
    yaxis='y2'
))

"""ADD APEX MARKERS"""
for apex in apex_distances:
    fig.add_vline(
        x=apex,
        line=dict(
            color='gray',
            width=.5,
            dash='dot'
        )
    )

"""ADD TURN NUMBERS"""
for apex, turn in zip(apex_distances, turn_numbers):
    fig.add_annotation(
        x=apex,
        y=max_speed + 10,
        text=f'T{turn}',
        showarrow=False,
        font=dict(color='white', size=12),
        xanchor='center',
        yanchor='bottom'
    )

"""SET PLOT LAYOUT"""
fig.update_layout(
    title=dict(
        text=f'{grand_prix} Grand Prix - {speed} - Lap {lap_number}',
        x=0.5,
        xanchor='center'
    ),
    xaxis_title='Distance (m)',
    yaxis_title='Speed (km/h)',
    height=1200,
    width = 3200,
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    legend=dict(title=f'Driver'),

    xaxis=dict(
        showgrid=True,
        gridcolor='gray',
        gridwidth=0.1,
        showspikes=True,
        spikecolor='white',
        spikemode='across',
        spikesnap='cursor',
        spikethickness=.5
    ),
    
    yaxis=dict(
        showgrid=True,
        gridcolor='gray',
        gridwidth=0.1,
        showspikes=True,
        spikecolor='white',
        spikemode='across',
        spikesnap='cursor',
        spikethickness=.5
    ),

    yaxis2=dict(
        title='Time Delta (s)',
        overlaying='y',
        side='right',
        showgrid=False,
        color='yellow'
    )

)

fig.show()